# **> Ajuste do formato da coluna `Pontuação Regata`**

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Banco de Súmulas - Sumulas.csv')

In [4]:
df.head()

,ID Resultado,ID Competidor,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Descarte,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição
0,NaN,NaN,Maximilian Maeder,5.0,Formula Kite Masc.,1.0,NaN,G,1,NaN,63.00,16.00,European Championship 2022
1,NaN,NaN,Maximilian Maeder,5.0,Formula Kite Masc.,(9.0),NaN,G,1,NaN,63.00,16.00,European Championship 2022
2,NaN,NaN,Maximilian Maeder,5.0,Formula Kite Masc.,5.0,NaN,G,1,NaN,63.00,16.00,European Championship 2022
3,NaN,NaN,Maximilian Maeder,5.0,Formula Kite Masc.,1.0,NaN,G,1,NaN,63.00,16.00,European Championship 2022
4,NaN,NaN,Maximilian Maeder,5.0,Formula Kite Masc.,1.0,NaN,G,1,NaN,63.00,16.00,European Championship 2022


In [5]:
# drop rows where Pontuação Regata is np.NaN
df = df.dropna(subset=['Pontuação Regata'])

In [6]:
df['Descarte'] = df['Descarte'].replace(np.NaN, None)
df['Punição'] = df['Punição'].replace(np.NaN, None)

**Casos de exemplo para tratamento:**

[x] 1.0

[x] 2.8

[x] 1

[x] -1

[x] (9.0)

[x] (9)

[x] (32.0 DNC)

[x] DSQ(69)

[x] DNF (32)

[x] (59.0 I BFD)

[x] (54.0 FL-SCP)

[x] DNF

[x] dnf lower case

[x] DNF 32

[x] RDG24

[x] RDG11.75

[x] 24.0 M-SCP

[x] 30.0 ARB

[x] 5.8 RDG F

[x] 4*BFD

[x] *18

In [8]:
def remove_parenthesis_or_asterisk(x):
    if x.__contains__('('):
        return x.replace('(', '').replace(')', '')
    elif x.__contains__('*'):
        return x.replace('*', '')

def is_number(x):
    try:
        float(x)
        return True
    except:
        return False
    
def update(column, current_value, new_value, df, position):
    if current_value == None:
        df.at[position, column] = new_value
    else:
        df.at[position, column] = current_value

In [9]:
PUNICOES = ['I BFD', 'FL-SCP', 'M-SCP', 'DNC', 'DNS', 
            'OCS', 'ZFP', 'UFD', 'BFD', 'SCP', 'NSC', 'DNF', 
            'RET', 'DSQ', 'DNE', 'RDG', 'DPI', 'ARB',
            'i bfd', 'fl-scp', 'm-scp', 'dnc', 'dns', 
            'ocs', 'zfp', 'ufd', 'bfd', 'scp', 'nsc', 'dnf', 
            'ret', 'dsq', 'dne', 'rdg', 'dpi', 'arb']

In [10]:
df_copy = df.copy()

for i, row in df.iterrows():
    pontos = row['Pontuação Regata']
    descarte = row['Descarte']
    punicao = row['Punição']
    
    if pontos.startswith('-'):
        """
        Caso do bug do cntrl+c e cntrl+v, em que o número entre parênteses se tornou um número negativo
        """
        pontos = pontos[1:]

    if is_number(pontos):
        """
        Se pontos for um número, i.e., não tiver qualquer outro texto associado à string.
        Na coluna de Descarte, devemos adicionar 0 se o valor atual for np.nan.
        Na coluna de Punição, devemos manter o valor atual.
        """
        pontos = float(pontos)
        # change in the dataset itself
        df_copy.at[i, 'Pontuação Regata'] = pontos
        update(column='Descarte', current_value=descarte, new_value=0, df=df_copy, position=i)
        #update(column='Punição', current_value=punicao, new_value=punicao, df=df, position=i)

    elif pontos.__contains__('(') or pontos.__contains__('*'):
        """
        Se pontos for uma string que contém um parênteses, pode ser um dos casos abaixo:
        (9.0), (32.0 DNC), DNF (32), DSQ(69).
        Nesse caso, se a remoção dos parênteses resultar em um número, devemos adicionar esse número
        na coluna de pontos. Se não, teremos que passar por mais alguns tratamentos de string.
        """
        pontos = remove_parenthesis_or_asterisk(pontos)
        
        if is_number(pontos):
            """
            Caso em que a remoção dos parênteses resulta apenas em um número.
            Nesse caso, houve descarte mas a indicação da punição não está na string ou não existe.
            Adicionamos 1 na coluna de Descarte, mas mantemos o valor atual na coluna de Punição.
            """
            pontos = float(pontos)
            df_copy.at[i, 'Pontuação Regata'] = pontos
            update(column='Descarte', current_value=descarte, new_value=1, df=df_copy, position=i)
            #update(column='Punição', current_value=punicao, new_value=punicao, df=df, position=i)

        else:
            """
            Casos em que a remoção dos parênteses não resulta em um número puro.
            """
            for p in PUNICOES:
                if pontos.__contains__(p):
                    """
                    Caso em que a string contém uma punição, i.e., DNC, DNS, etc.
                    Nesse caso, removemos a punição da string, transformamos o valor em float e adicionamos
                    a pontuação na coluna de pontos. Adicionamos 1 na coluna de Descarte e adicionamos a punição
                    na coluna de Punição.
                    """
                    pontos = pontos.replace(p, '')
                    try:
                        pontos = float(pontos)
                        df_copy.at[i, 'Pontuação Regata'] = pontos
                    except:
                        df_copy.at[i, 'Pontuação Regata'] = 'flag'
                    update(column='Descarte', current_value=descarte, new_value=1, df=df_copy, position=i)
                    update(column='Punição', current_value=punicao, new_value=p.upper(), df=df_copy, position=i)
                    break
            
    else:
        """
        Casos em que pontos não é um número e não possui parênteses (não há descarte), 
        mas há uma punição na string.
        """
        for p in PUNICOES:
            if pontos.__contains__(p):
                """
                Caso em que a string contém uma punição, i.e., DNC, DNS, etc.
                Nesse caso, removemos a punição da string, transformamos o valor em float e adicionamos
                a pontuação na coluna de pontos. Adicionamos 0 na coluna de Descarte e adicionamos a punição
                na coluna de Punição.
                """
                pontos = pontos.replace(p, '')
                try:
                    pontos = float(pontos)
                    df_copy.at[i, 'Pontuação Regata'] = pontos
                except:
                    df_copy.at[i, 'Pontuação Regata'] = 'flag'
                update(column='Descarte', current_value=descarte, new_value=0, df=df_copy, position=i)
                update(column='Punição', current_value=punicao, new_value=p.upper(), df=df_copy, position=i)
                break

In [11]:
# join df and df_copy
df_copy = df.join(df_copy[['Pontuação Regata', 'Descarte', 'Punição']], rsuffix='_ajustada')

# filter columns
columns = ['Pontuação Regata', 'Pontuação Regata_ajustada', 'Descarte', 'Descarte_ajustada', 'Punição', 'Punição_ajustada']
df_copy = df_copy[columns]

# sample rows
df_copy.sample(50000)

df_copy.to_excel('pontuacao_ajustada.xlsx', index=False)

## **> Obtenção do dataset final**

In [12]:
# # colunas que vão entrar no dataframe final
# columns_to_model = ['Nome Competidor', 'Classe Vela', 'Flotilha', 'Posição Geral', 'Nett', 'Nome Competição']
# df_to_model = df[columns_to_model]

# # dropa as repetições regata a regata (mantém a primeira)
# df_to_model.drop_duplicates(keep='first', inplace=True)

# df_to_model.head()